In [1]:
!pip3 install  -r requirements.txt

In [2]:
from dotenv import dotenv_values
import openai, os
import numpy as np
from numpy.linalg import norm

secrets= dotenv_values(".env")

In [3]:
os.environ['OPENAI_API_KEY'] = secrets['OPENAI_API_KEY']

In [6]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

file_path = '/Users/rayanaay/Desktop/projects/langchain/summarizing_project/mix_data/rust_essentials.pdf'

loader = PyPDFLoader(file_path=file_path,)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0
)

data = loader.load_and_split(text_splitter=text_splitter)
data[:5]

[Document(page_content='Rust Essentials', metadata={'source': '/Users/rayanaay/Desktop/projects/langchain/summarizing_project/mix_data/rust_essentials.pdf', 'page': 1}),
 Document(page_content='Second Edition\n \n \n \n \n \n \n \n \n \nA quick guide to writing fast, safe, and concurrent systems\nand applications', metadata={'source': '/Users/rayanaay/Desktop/projects/langchain/summarizing_project/mix_data/rust_essentials.pdf', 'page': 2}),
 Document(page_content='Ivo Balbaert\n \n \n \n \n \n \nBIRMINGHAM - MUMBAI', metadata={'source': '/Users/rayanaay/Desktop/projects/langchain/summarizing_project/mix_data/rust_essentials.pdf', 'page': 3}),
 Document(page_content='Rust Essentials', metadata={'source': '/Users/rayanaay/Desktop/projects/langchain/summarizing_project/mix_data/rust_essentials.pdf', 'page': 4}),
 Document(page_content='Second Edition\nCopyright © 2017 Packt Publishing\n \nAll rights reserved. \nNo part of this book may be reproduced, stored in a retrieval\nsystem, or tran

In [14]:
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = "text-embedding-3-small"  
embeddings = OpenAIEmbeddings(model=model_name, show_progress_bar=True)

In [13]:
# Initialize the vector database
from langchain.vectorstores import Chroma

index = Chroma.from_documents(
    data,
    embedding=embeddings
)

/Users/rayanaay/anaconda3/envs/langchain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 9/9 [00:16<00:00,  1.79s/it]


In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StdOutCallbackHandler

llm = ChatOpenAI()

# The Stuff Chain

- Combine relevant documents then feed them into the LLM.

- It works by chunking all the relevant retrieved documents, processed individually, and then combining the summaries to generate a final summary

- The context is the documents retrieved from the LLM used to answer the prompt.

In [ ]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=index.as_retriever(),
    chain_type="stuff",
    verbose=True
)

# Map-Reduce Chain

- Different from the Stuff Strategy. It does not combine relevant documents, but combine relevant information extracted using the LLM first, then combine the relevant informations used as a context to the LLM that will answer the user question.
- The combined context should be equal or smaller than the LLM's context window
- This method is obviously more expansive, because in the first step the LLM needs to go over all documents one by one.

In [ ]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=index.as_retriever(),
    chain_type="map_reduce",
    verbose=True
)

# Refine Chain

- step1: we ask the LLM to answer the question from the initial document.

- step2: We then ask the LLM to answer the same question using the second document but with an initial context from the initial answer, we got then a **refined answer**

- step3: Iterate step 2 until the provided context does not offer any new information about the user question, the original answer adequately explains and answer the questions. Therefore no refinement is necessary based on the given context.

In [ ]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=index.as_retriever(),
    chain_type="refine",
    verbose=True
)

# Map-Rerank Chain

- Use an LLM to generate a score on how well the question has been answered, return the argmax answer as the final answer.
- Sometimes, the LLM will break the chain because it will not respect the template set by the strategy, instead of returning a score it will return something else. 

In [ ]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=index.as_retriever(),
    chain_type="map_rerank",
    verbose=True
)